# Project: Multi-Agent Supervisor

This is an example of using a supervisor agent that acts as a router for user requests. The supervisor determines whick agent can fullfil a request, examine the agents response, and determines if the request is filled or another agent needs to be involved.

Install the required packages.

In [2]:
!pip install -q -r requirements.txt --progress-bar off

Provide a OpenAI key.

In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Provide OPENAI_API_KEY")

Provide OPENAI_API_KEY··········



Create the Agent Supervisor.

In [9]:
from AgentSupervisor import AgentSupervisor
import logging as log
log.getLogger().setLevel(log.WARN)

def print_response(response):
  for idx, message in enumerate(response):
    print(f"{idx+1}. {message}")
    print("---")

agent_supervisor = AgentSupervisor()

Lets test the supervisor. This supervisor only has two simple agents:

1. An agent to write salutations to family members.
2. An agent to retrieve the NYC reservoir capacity utilization.

The first request is for the NYC water levels.


In [10]:
request = "Retreieve the NYC reservior levels."
response = agent_supervisor.make_request(request)
print_response(response)

1. Retreieve the NYC reservior levels.
---
2. ** The NYC reservior levels are as follows **
- Normal Capacity: 97.7%
- Current Capacity: 99.6%
---


This looks correct. The supervisor sent the request to the correct agent. The agent is returning a hard coded response for this request.

Now lets write a salutation.

In [11]:
request="Create a salutaion for my mother."
response = agent_supervisor.make_request(request)
print_response(response)

1. Create a salutaion for my mother.
---
2. ** Here is the Salutation **

Salutation: Hey mom, love you always!
Length: 25 characters
---


This looks correct. The supervisor sent the request to the correct agent which used one of its tools to write a salutation using an LLM and another to get the length of that salutation which just used the Python length function.

Now lets send two requests.

In [12]:
request="Write a salutation to my wife. Then retreieve the NYC reservior levels."
response = agent_supervisor.make_request(request)
print_response(response)

1. Write a salutation to my wife. Then retreieve the NYC reservior levels.
---
2. ** Here is the Salutation **

Hey love bug, miss you!

Length: 23 characters
---
3. ** The NYC reservior levels are as follows **

- Normal Capacity: 97.7%
- Current Capacity: 99.6%
---


This also looks correct.

While experimenting with this service, I noticed that sometimes the LLM used to route requests would get confused and send requests to the wrong agent. Especially when multiple requests are made. This is to be expected as results of an LLM invocation are not determinant. This can be addresed by breaking up a request into its parts.   

Below is an example of using an LLM to split the request into a list and then submitting them to the service individually.

In [23]:
import json
import util

def get_list_from(txt: str) -> str:
  response = util.get_llm_model().invoke(input=("system: You extracts requests from text and summarize"
                                                f" them into lists. Summarize the requests below into a list.\nhuman: {txt}")).content
  return response

def get_json_from(txt: str) -> str:
  response = util.get_llm_model().invoke(input=("system: You transform lists into a JSON document."
                                                " Transform the list below into a JSON document with"
                                                f" a single element called requests.\n{txt}" )).content
  return response

def json_to_dictionary(_json: str) -> dict:
  return json.loads(_json)

request="Write a salutation to my wife. Then retreieve the NYC reservior levels."
list_response = get_list_from(request)
print(list_response)
json_response = get_json_from(list_response)
print(json_response)
requests = json_to_dictionary(json_response)["requests"]
print(requests)

1. Write a salutation to wife
2. Retrieve NYC reservoir levels
{
  "requests": [
    "Write a salutation to wife",
    "Retrieve NYC reservoir levels"
  ]
}
['Write a salutation to wife', 'Retrieve NYC reservoir levels']


This looks correct. Now submit the requests one at a time.

In [24]:
responses = []

for request in requests:
  response = agent_supervisor.make_request(request)
  responses.append(response)

for idx, response in enumerate(responses):
  print(f"{idx+1}: {response}")
  print("---")

1: ['Write a salutation to wife', '** Here is the Salutation **\n\nSalutation to wife: Hey love bug, miss you!\nLength: 23 characters']
---
2: ['Retrieve NYC reservoir levels', '** The NYC reservior levels are as follows **\n- Normal Capacity: 97.7%\n- Current Capacity: 99.6%']
---


This looks correct.